In [2]:
!pip install torch

In [0]:
import torch
import random
import numpy as np
import pandas as pd
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.backends.cudnn.deterministic = True

In [0]:
from google.colab import drive
path = '/content/drive/My Drive/dataset'

In [5]:
!ls '/content/drive/My Drive/dataset'

test  train


In [0]:
import torch.nn as nn
class PhysioNet(nn.Module):

    def __init__(self):
        super(PhysioNet, self).__init__()

        out = (4, 10, 48)
        kernel = (5, 4, 5)
        padding = (1, 1, 0)
        stride = (1, 1, 2)
        out_size = 5
        fc_out1 = 20
        fc_out2 = 10
        out_classes = 1
        self.batch_norm0 = torch.nn.BatchNorm2d(1)
        self.conv1 = torch.nn.Conv2d(in_channels=1, out_channels=out[0], kernel_size=kernel[0], padding=padding[0], stride=stride[0])
        self.act1 = torch.nn.LeakyReLU()
        self.batch_norm1 = torch.nn.BatchNorm2d(out[0])
        #self.pool1 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        '''
        self.conv2 = torch.nn.Conv2d(in_channels=out[0], out_channels=out[1], kernel_size=kernel[1], padding=padding[1], stride=stride[1])
        self.act2 = torch.nn.ReLU()
        self.batch_norm2 = torch.nn.BatchNorm2d(out[1])
        self.pool2 = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        '''
        #self.conv3 = torch.nn.Conv2d(in_channels=out[1], out_channels=out[2], kernel_size=kernel[2], padding=padding[2], stride=stride[2])
        #self.act3 = torch.nn.ReLU()
        #self.batch_norm3 = torch.nn.BatchNorm2d(out[2])
        #self.pool3 = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        #self.fc1 = torch.nn.Linear(30*10 * out[0], fc_out1)
        self.fc1 = torch.nn.Linear(62*19 * out[0], fc_out1)
        self.act4 = torch.nn.LeakyReLU()
        
        self.fc2 = torch.nn.Linear(fc_out1, fc_out2)
        self.act5 = torch.nn.LeakyReLU()
        
        self.fc3 = torch.nn.Linear(fc_out2, out_classes)
    
    def forward(self, x):
        x = self.batch_norm0(x)
        print("x = ", x)
        x = self.conv1(x)
        print("conv = ", x)
        x = self.act1(x)
        x = self.batch_norm1(x)
        #x = self.pool1(x)
        '''
        x = self.conv2(x)
        x = self.act2(x)
        x = self.batch_norm2(x)
        x = self.pool2(x)
        '''

        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        x = self.act5(x)
        x = self.fc3(x)
        return x

In [0]:
physionet = PhysioNet()

In [92]:
from torchsummary import summary
summary(physionet, (1, 64, 21))

x =  tensor([[[[-0.4048,  0.2041, -0.7590,  ...,  1.1974, -1.4449,  1.1739],
          [-1.7182,  0.2444, -0.6769,  ..., -1.6642, -0.1996, -0.4123],
          [-1.3143,  1.2327, -0.7666,  ..., -0.0152,  0.5769, -0.8350],
          ...,
          [ 0.5866,  0.1597, -0.1823,  ...,  0.9048, -1.3515,  1.0013],
          [-1.6673,  0.4075, -0.0919,  ...,  0.4941,  1.5089,  1.3128],
          [ 1.4426,  1.2103, -1.1079,  ...,  1.6558, -0.9191,  1.6572]]],


        [[[ 1.5785, -0.4321, -0.9612,  ..., -0.0120, -1.3765,  1.4619],
          [ 0.1874, -1.0104, -0.4644,  ..., -0.9277,  0.7642, -1.5761],
          [-0.5248, -1.0873,  0.1089,  ..., -0.2136,  0.0457,  1.1864],
          ...,
          [ 0.8358,  0.7866, -0.2455,  ..., -0.8417, -1.3409, -0.7828],
          [-0.1804, -0.4957, -1.5358,  ...,  0.6514,  1.3513,  0.4444],
          [ 1.6339,  0.2289, -0.2023,  ...,  0.9423,  0.4204,  0.3088]]]],
       grad_fn=<NativeBatchNormBackward>)
conv =  tensor([[[[ 4.8920e-01,  2.6724e-01,  8.5539

In [0]:
Y_train = pd.read_csv("/content/drive/My Drive/dataset/train/y.csv", delimiter=";")
Y_train = np.array(Y_train.drop(["0.5010000000000012"],axis=1))


In [0]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
physionet = physionet.to(device)
#for param in physionet.parameters():
#    param.requires_grad = False

In [0]:
loss = torch.nn.MSELoss()
optimizer = torch.optim.Adam(physionet.parameters(), lr=1.0e-3)

In [96]:
x = np.array(pd.read_csv("/content/drive/My Drive/dataset/test/{}.csv".format(1), delimiter=";"))[:-1]
print(x.shape)
x = np.delete(x.reshape(64,24),23,1)
x = np.delete(x.reshape(64,23),22,1)
x = np.delete(x.reshape(64,22),21,1)
print(x.shape)

(64, 24)
(64, 21)


In [0]:
y_test = pd.read_csv("/content/drive/My Drive/dataset/test/y.csv", delimiter=";")
y_test = np.array(y_test.drop(["0.7650000000001"],axis=1))
X_test = []
for i in range(len(y_test)):
  x = pd.read_csv("/content/drive/My Drive/dataset/test/{}.csv".format(i), delimiter=";")
  x = np.array(x)[:-1]
  x = np.delete(x.reshape(64,24),23,1)
  x = np.delete(x.reshape(64,23),22,1)
  x = np.delete(x.reshape(64,22),21,1)
  X_test.append(x)

In [98]:
test_accuracy_history = []
test_loss_history = []


len_x_train = len(Y_train)
batch_size = 100
for epoch in range(10000):
    order = np.random.permutation(len_x_train)
    for start_index in range(0, len_x_train, batch_size):
        optimizer.zero_grad()
        
        batch_indexes = order[start_index:start_index+batch_size]
        X_train = []
        for file in batch_indexes:
          x = np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(file), delimiter=";"))[:-1]
          x = np.delete(x.reshape(64,24),23,1)
          x = np.delete(x.reshape(64,23),22,1)
          x = np.delete(x.reshape(64,22),21,1)
          X_train.append(x)
        #X_train = torch.DoubleTensor(np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(1), delimiter=";"))[:-1])
        preds = physionet.forward(torch.tensor(X_train).unsqueeze(1).float())
        y_batch = []
        for file in batch_indexes:
          y_batch.append(Y_train[file])

        loss_value = loss(preds.float(), torch.tensor(y_batch).float())
        loss_value.backward()
        
        optimizer.step()
    
    test_preds = physionet.forward(torch.tensor(X_test).unsqueeze(1).float())
    print(test_preds)
    test_loss_history.append(loss(test_preds.float(), torch.tensor(y_test).float()))#.data.cpu())
    
    accuracy = (test_preds.argmax(dim=1) == y_test).float().mean().data.cpu()
    test_accuracy_history.append(accuracy)
    
    print(accuracy)

Streaming output truncated to the last 5000 lines.
            2.0442e-01,  6.0011e-02]],

         [[-5.1929e-02, -4.6418e-02, -1.5348e-01,  ..., -1.0118e-01,
            8.0791e-02,  7.6237e-02],
          [-5.4001e-02, -7.1588e-02, -1.8471e-01,  ..., -5.1311e-02,
            1.6044e-01,  9.7797e-02],
          [-4.8443e-02, -6.6147e-02, -1.7995e-01,  ..., -4.0857e-02,
            1.7347e-01,  1.0073e-01],
          ...,
          [-4.7095e-02, -7.8966e-02, -1.6674e-01,  ..., -2.1543e-02,
            1.8125e-01,  1.0182e-01],
          [-5.2985e-02, -7.7740e-02, -1.6646e-01,  ..., -1.4552e-02,
            1.8789e-01,  9.3853e-02],
          [-8.0301e-02, -1.0195e-01, -1.6190e-01,  ...,  2.7816e-02,
            2.0563e-01,  3.9991e-02]]]], grad_fn=<MkldnnConvolutionBackward>)
x =  tensor([[[[ 1.5962e-01, -1.0622e-02, -1.2585e-01,  ...,  5.9204e-01,
            1.4150e-01,  5.2168e-02],
          [ 1.3503e-01, -3.2631e-02, -1.3232e-01,  ...,  5.8362e-01,
            1.3503e-01,  3.6191

AttributeError: ignored

In [0]:

X_train = torch.FloatTensor(np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(1), delimiter=";")))

In [0]:
X_train = []
for file in batch_indexes:
  x = np.double(np.array(pd.read_csv("/content/drive/My Drive/dataset/train/{}.csv".format(file), delimiter=";"), dtype=np.double)[:-1])
  X_train.append(x)


In [0]:
torch.DoubleTensor(X_train).unsqueeze(1).float().shape

In [0]:
100*1364*1200 *20

In [0]:
batch_indexes = order[0:0+100]

y_batch = []
for file in batch_indexes:
  y_batch.append(Y_train[file])